In [102]:
from datetime import datetime

In [107]:
import argparse
import socket
import sys

"""
Simple example pokerbot, written in python.

This is an example of a bare bones pokerbot. It only sets up the socket
necessary to connect with the engine and then always returns the same action.
It is meant as an example of how a pokerbot should communicate with the engine.
"""
class Johnny:
    """
    self.bot_name = bot_name in config file. Corresponds to PLAYER_X_NAME field.
    
    self.hand = a dictionary storing properties of the current hand. Has the following keys.
        hand_id = int. ID of hand.
        button = boolean. Are we the dealer?
        hole1 = hole card 1
        hole2 = hole card 2
        board = current known board cards 
        action_history = a list of the last actions. action_history[i] = another list that describes
                         the previous actions up until action point i.
        result = int. Chips won (can be a negative integer if we lost chips)
    
    self.state = A dictionary representing the our state in the current match. Has the following keys.
        my_bank = our current bankroll 
        their_bank = their current bankroll
        time_bank = cumulative time remaining in the match
        
        
    """
    
    
    def __init__(self, bot_name):
        self.bot_name = bot_name
        self.hand = {}
        self.hand['action_history'] = []
        self.hand['pot_size'] = []
        
        self.state = {}
        
    
    def parse_data(self, data):
        splits = data.split()
        packet_type = splits[0]
        print packet_type
        
        if packet_type == "NEWHAND":
            self.parse_new_hand(splits)
        elif packet_type == "GETACTION":
            self.parse_get_action(splits)
        elif packet_type == "HANDOVER":
            self.parse_hand_over(splits)
            
    def parse_win_result(self, wr):
        splits = wr.split(":")
        amt = int(splits[1])
        winner = splits[2]
        
        if winner == self.bot_name:
            return amt
        else:
            return -amt
        
            
    def parse_hand_over(self, data_splits):
        # HANDOVER Stack1 Stack2 numBoardCards [boardCards] numLastActions [lastActions] timeBank
        # Can ignore Stack1 and Stack2 because we'll get it on the next NEWHAND packet.
        num_board_cards = int(data_splits[3])
        counter = 4
        self.hand['board'] = data_splits[counter:counter+num_board_cards]
        counter += num_board_cards
        
        num_last_actions = int(data_splits[counter])
        counter += 1
        self.hand['action_history'].append(data_splits[counter:counter+num_last_actions-1])
        self.hand['result'] = self.parse_win_result(data_splits[counter+num_last_actions-1])
        counter += num_last_actions
        
        self.state['time_bank'] = float(data_splits[-1])
        
            
    def parse_get_action(self, data_splits):
        # GETACTION potSize numBoardCards [boardCards] numLastActions [lastActions] numLegalActions [legalActions] timebank
        self.hand['pot_size'].append(int(data_splits[1]))
        
        num_board_cards = int(data_splits[2])
        counter = 3
        self.hand['board'] = data_splits[counter:counter+num_board_cards]
        counter += num_board_cards
        
        num_last_actions = int(data_splits[counter])
        counter += 1
        self.hand['action_history'].append(data_splits[counter:counter+num_last_actions])
        counter += num_last_actions
        
        num_legal_actions = int(data_splits[counter])
        counter += 1
        self.possible_actions = data_splits[counter:counter+num_legal_actions]
        counter += num_legal_actions
        
        self.state['time_bank'] = float(data_splits[-1])

    def parse_new_hand(self, data_splits):
        # NEWHAND handId button holeCard1 holeCard2 myBank otherBank timeBank
        self.hand['hand_id'] = int(data_splits[1])
        self.hand['button'] = data_splits[2]
        self.hand['hole1'] = data_splits[3]
        self.hand['hole2'] = data_splits[4]
        
        self.state['my_bank'] = int(data_splits[5])
        self.state['their_bank'] = int(data_splits[6])
        self.state['time_bank'] = float(data_splits[7])
    
    def run(self, input_socket):
        # Get a file-object for reading packets from the socket.
        # Using this ensures that you get exactly one packet per read.
        f_in = input_socket.makefile()
        while True:
            # Block until the engine sends us a packet.
            data = f_in.readline().strip()
            # If data is None, connection has closed.
            if not data:
                print "Gameover, engine disconnected."
                break

            # Here is where you should implement code to parse the packets from
            # the engine and act on it. We are just printing it instead.
            self.parse_data(data)
            
            # Parse data
            # Build features
            # Take action 

            # When appropriate, reply to the engine with a legal action.
            # The engine will ignore all spurious responses.
            # The engine will also check/fold for you if you return an
            # illegal action.
            # When sending responses, terminate each response with a newline
            # character (\n) or your bot will hang!
            word = data.split()[0]
            if word == "GETACTION":
                # Currently CHECK on every move. You'll want to change this.
                s.send("CHECK\n")
            elif word == "REQUESTKEYVALUES":
                # At the end, the engine will allow your bot save key/value pairs.
                # Send FINISH to indicate you're done.
                s.send("FINISH\n")
        # Clean up the socket.
        s.close()

In [108]:
# data = []
# data.append("NEWHAND 98 false 9c 4s -111 111 9.931115")
# data.append("GETACTION 4 0 3 POST:1:P2 POST:2:P1 CALL:P2 2 CHECK RAISE:4:200 9.931115124999993")
# data.append("GETACTION 4 3 4d 5c Js 2 CHECK:P1 DEAL:FLOP 3 CHECK DISCARD:9c DISCARD:4s 9.930735672999992")
# data.append("GETACTION 4 3 4d 5c Js 2 CHECK:P1 CHECK:P2 2 CHECK BET:2:198 9.930290227999992")
# data.append("GETACTION 4 4 4d 5c Js Jh 3 CHECK:P1 CHECK:P2 DEAL:TURN 3 CHECK DISCARD:9c DISCARD:4s 9.929636328999992")
# data.append("GETACTION 4 4 4d 5c Js Jh 2 CHECK:P1 DISCARD:P2 2 CHECK BET:2:198 9.928944276999992")
# data.append("GETACTION 83 4 4d 5c Js Jh 2 CHECK:P1 BET:79:P2 3 FOLD CALL RAISE:158:198 9.928641209999991")
# data.append("HANDOVER -113 113 4 4d 5c Js Jh 3 FOLD:P1 REFUND:79:P2 WIN:4:P2 9.928236565999992")


data = []
data.append("NEWHAND 80 false Kd Qh -77 77 9.942149")
data.append("GETACTION 4 0 3 POST:1:P2 POST:2:P1 CALL:P2 2 CHECK RAISE:4:200 9.942148733999993")
data.append("GETACTION 4 3 9d 4h Ts 2 CHECK:P1 DEAL:FLOP 3 CHECK DISCARD:Kd DISCARD:Qh 9.941793586999992")
data.append("GETACTION 4 3 9d 4h Ts 2 CHECK:P1 CHECK:P2 2 CHECK BET:2:198 9.941348511999992")
data.append("GETACTION 56 3 9d 4h Ts 2 CHECK:P1 BET:52:P2 3 FOLD CALL RAISE:104:198 9.940922597999993")
data.append("HANDOVER -79 79 3 9d 4h Ts 3 FOLD:P1 REFUND:52:P2 WIN:4:P2 9.940476787999993")



#data = "GETACTION 4 3 5d Kd 6h 2 CHECK:P1 DEAL:FLOP 3 CHECK DISCARD:Jc DISCARD:9s 9.995735496999998"

In [109]:
jp = Johnny("P1")

In [110]:
startTime = datetime.now()
for i in range(len(data)):
    jp.parse_data(data[i])
    
print datetime.now() - startTime

NEWHAND
GETACTION
GETACTION
GETACTION
GETACTION
HANDOVER
0:00:00.000659


In [111]:
print jp.bot_name
print jp.state
jp.hand

P1
{'my_bank': -77, 'their_bank': 77, 'time_bank': 9.940476787999993}


{'action_history': [['POST:1:P2', 'POST:2:P1', 'CALL:P2'],
  ['CHECK:P1', 'DEAL:FLOP'],
  ['CHECK:P1', 'CHECK:P2'],
  ['CHECK:P1', 'BET:52:P2'],
  ['FOLD:P1', 'REFUND:52:P2']],
 'board': ['9d', '4h', 'Ts'],
 'button': 'false',
 'hand_id': 80,
 'hole1': 'Kd',
 'hole2': 'Qh',
 'pot_size': [4, 4, 4, 56],
 'result': -4}

NameError: name 'attributes' is not defined

In [ ]:
jp.time